In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-07 18:52:17.213997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746658337.304498  359027 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746658337.334698  359027 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746658337.444275  359027 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746658337.444333  359027 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746658337.444335  359027 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:

# Parameters for Inference
heuristic_params = {"ego_distance": 0.5, "cardinality": 1} # Hueristics and their weight parameters (to be inferred)

construal_count_baseline = 2 # Number of construals to sample for baseline data generation
trajectory_count_baseline = 2 # Number of baseline trajectories to generate per construal



### Specify Environment Configuration ###

# |Location to store (and find pre-computed) simulation results
simulation_results_path = "examples/CoDec_Research/results/simulation_results/"
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]

# |Location to store simulation results
out_dir = "examples/CoDec_Research/results/simulation_results/"

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal'

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
num_parallel_envs = 2
total_envs = 4
device = "cpu" # cpu just because we're in a notebook
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1      # Agents observed except self (used for vector sizes)
sample_size_utility = 1                             # Number of samples to calculate expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )


In [3]:

### Select Construals for Baseline Data ###
scene_constr_dict = None

#2# |Generate Construal Execution Values through simulator sampling
default_values = None

#2# |Check if saved data is available
for srFile in simulation_results_files:
    if "construal_vals" in srFile:
        with open(srFile, 'rb') as opn_file:
            default_values = pickle.load(opn_file)
    else:
        continue

if default_values is None:
    default_values, traj_obs, ground_truth = generate_all_construal_trajnval(out_dir=out_dir,
                                                                                sim_agent=sim_agent,
                                                                                observed_agents_count=observed_agents_count,
                                                                                construal_size=construal_size,
                                                                                num_parallel_envs=num_parallel_envs,
                                                                                max_agents=max_agents,
                                                                                sample_size=sample_size_utility,
                                                                                device=device,
                                                                                train_loader=train_loader,
                                                                                env=env,
                                                                                env_multi_agent=env_multi_agent,
                                                                                generate_animations=False)

#2# |Generate Construal Heuristic Values (Heuristic 1: Distance from ego)
heuristic_values = get_constral_heurisrtic_values(env, train_loader, default_values, heuristic_params=heuristic_params)

#2# |Sample from construal values
def sample_construals(heuristic_values: dict, sample_count: int) -> dict:
    """
    Sample construals based on heuristic values.
    """
    sampled_construals = {}
    for scene_name, construal_info in heuristic_values.items():
        constr_indices, constr_values = zip(*construal_info.items())
        sampled_indices = torch.multinomial(torch.tensor(constr_values), num_samples=sample_count, \
                                                replacement=False).tolist()
        sampled_construals[scene_name] = {constr_indices[i]: constr_values[i] for i in sampled_indices}
        print(f"Sampled construals for scene {scene_name}: {sampled_construals[scene_name].keys()}")

    return sampled_construals

scene_constr_dict = sample_construals(heuristic_values, sample_count=construal_count_baseline)




Processing Waymo batches:   0%|          | 0/2 [00:00<?, ?it/s]

Indices of all moving vehicles (by scene):  [(0, 1, 5, 9, 10, 42, 43), (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17)]
	sample  1
		Step: 90
Processed masks:  ((0,), (0,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((1,), (1,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((5,), (2,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((9,), (3,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((10,), (4,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((42,), (5,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((43,), (7,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((0, 1, 5, 9, 10, 42, 43), (8,)) , with values: [1.0, 0.0]
	sample  1
		Step: 90
Processed masks:  ((0, 1, 5, 9, 10, 42, 43), (9,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((0, 1, 5, 9, 10, 42, 43), (10,)) , with values: [1.0, 1.0]
	sample  1
		S

Processing Waymo batches:  50%|█████     | 1/2 [00:42<00:42, 42.71s/it]

Indices of all moving vehicles (by scene):  [(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20), (0, 7, 59)]
	sample  1
		Step: 91
Processed masks:  ((0,), (0,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((1,), (7,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((2,), (59,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((3,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((4,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((5,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((6,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((7,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((8,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((9,), (0, 7, 59)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed ma

Processing Waymo batches: 100%|██████████| 2/2 [01:29<00:00, 44.63s/it]


Processed masks:  ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20), (0, 7, 59)) , with values: [1.0, 1.0]
Baseline data saved to:  examples/CoDec_Research/results/simulation_results/construal_vals_2025-05-07 18:53:50.775437.pickle


Baseline data saved to:  examples/CoDec_Research/results/simulation_results/all_constr_obs_2025-05-07 18:53:50.776554.pickle
Baseline data saved to:  examples/CoDec_Research/results/simulation_results/ground_truth_2025-05-07 18:53:50.800582.pickle


Processing Waymo batches: 100%|██████████| 2/2 [00:00<00:00,  8.61it/s]

Sampled construals for scene tfrecord-00101-of-01000_71: dict_keys([(10,), (42,)])
Sampled construals for scene tfrecord-00117-of-01000_240: dict_keys([(16,), (17,)])
Sampled construals for scene tfrecord-00143-of-01000_179: dict_keys([(1,), (6,)])
Sampled construals for scene tfrecord-00144-of-01000_130: dict_keys([(0,), (59,)])


In [4]:
heuristic_values

{'tfrecord-00101-of-01000_71': {(0,): 0.17234168642266534,
  (1,): 0.16472567120256412,
  (5,): 0.11892531864484873,
  (9,): 0.11259284461628764,
  (10,): 0.15660503433548612,
  (42,): 0.11704213354393044,
  (43,): 0.10864376279363638,
  (0, 1, 5, 9, 10, 42, 43): 0.04912354844058117},
 'tfrecord-00117-of-01000_240': {(0,): 0.07688420193601221,
  (1,): 0.05819550970835978,
  (2,): 0.048447271129240826,
  (3,): 0.05194195826656496,
  (4,): 0.07241930159752175,
  (5,): 0.05698325131513191,
  (7,): 0.07230305858181311,
  (8,): 0.0195775648610658,
  (9,): 0.05956666895360849,
  (10,): 0.055649903895327896,
  (11,): 0.06128659291389575,
  (12,): 0.06468634438058114,
  (13,): 0.05738199365026118,
  (14,): 0.062445301620712775,
  (15,): 0.06349093422812116,
  (16,): 0.05041805793912688,
  (17,): 0.04663262572172881,
  (0,
   1,
   2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17): 0.021689459300925626},
 'tfrecord-00143-of-01000_179': {(0,): 0.05280

In [5]:
### Generate Synthetic Ground Truth for Selected Construals (Baseline Data on Which to Perform Inference) ###
state_action_pairs = None

# |Check if saved data is available
for srFile in simulation_results_files:
    if "baseline_state_action_pairs" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        state_action_pairs = pickle.load(opn_file)

if state_action_pairs is None:
    state_action_pairs = generate_baseline_data(out_dir=out_dir,
                                                sim_agent=sim_agent,
                                                num_parallel_envs=num_parallel_envs,
                                                max_agents=max_agents,
                                                sample_size=trajectory_count_baseline,
                                                device=device,
                                                train_loader=train_loader,
                                                env=env,
                                                env_multi_agent=env_multi_agent,
                                                observed_agents_count=observed_agents_count,
                                                construal_size=construal_size,
                                                selected_construals=scene_constr_dict,
                                                generate_animations=False)


Processing Waymo batches:   0%|          | 0/2 [00:00<?, ?it/s]

	sample  1
		Step: 90
	sample  2
		Step: 90
Processed masks:  ((10,), (16,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
	sample  2
		Step: 90

Processing Waymo batches:  50%|█████     | 1/2 [00:09<00:09,  9.12s/it]


Processed masks:  ((42,), (17,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
	sample  2
		Step: 91
Processed masks:  ((1,), (0,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
	sample  2
		Step: 91
Processed masks:  ((6,), (59,)) , with values: [1.0, 1.0]


Processing Waymo batches: 100%|██████████| 2/2 [00:17<00:00,  8.92s/it]


Baseline data saved to:  examples/CoDec_Research/results/simulation_results/baseline_state_action_pairs_2025-05-07 18:54:08.931512.pickle


In [6]:
### Compute Construal Log Likelihoods ###
construal_action_likelihoods = None

# |Check if saved data is available
for srFile in simulation_results_files:
    if "log_likelihood_measures" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        construal_action_likelihoods = pickle.load(opn_file)

if construal_action_likelihoods is None:
    construal_action_likelihoods = evaluate_construals(state_action_pairs, construal_size, sim_agent, out_dir, device=device)

# |Clear memory for large variable, once it has served its purpose
# del state_action_pairs




Processing Scene:  tfrecord-00101-of-01000_71
(10,) (0,)
Processing baseline construal (10,) against construal (0,), sample 0
Processing baseline construal (10,) against construal (0,), sample 1
(10,) (1,)
Processing baseline construal (10,) against construal (1,), sample 0
Processing baseline construal (10,) against construal (1,), sample 1
(10,) (5,)
Processing baseline construal (10,) against construal (5,), sample 0
Processing baseline construal (10,) against construal (5,), sample 1
(10,) (9,)
Processing baseline construal (10,) against construal (9,), sample 0
Processing baseline construal (10,) against construal (9,), sample 1
(10,) (10,)
Processing baseline construal (10,) against construal (10,), sample 0
Processing baseline construal (10,) against construal (10,), sample 1
(10,) (42,)
Processing baseline construal (10,) against construal (42,), sample 0
Processing baseline construal (10,) against construal (42,), sample 1
(10,) (43,)
Processing baseline construal (10,) agains

In [7]:

### Sanity Check ###
scene_constr_dict = None
scene_constr_diff_dict = None

# |Check if saved data is available
for srFile in simulation_results_files:
    if "highest_construal_dict_log_likelihood_diff" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_diff_dict = pickle.load(opn_file)
    if "highest_construal_dict_log_likelihood" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_dict = pickle.load(opn_file)

if scene_constr_dict is None:
    scene_constr_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir)
if scene_constr_diff_dict is None:
    scene_constr_diff_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir, likelihood_key="log_likelihood_diff")


Highest scene construals dict saved to:  examples/CoDec_Research/results/simulation_results/highest_construal_dict_log_likelihood_2025-05-07 18:54:52.679450.pickle
Highest scene construals dict saved to:  examples/CoDec_Research/results/simulation_results/highest_construal_dict_log_likelihood_diff_2025-05-07 18:54:52.682040.pickle


In [8]:
for scene_name_, scene_info_ in construal_action_likelihoods.items():
    print(f"Scene: {scene_name_}")
    for base_construal_name_, base_construal_info_ in scene_info_.items():
        print_dict = {}
        for test_construal_name_, test_construal_info_ in base_construal_info_.items():
            for sample_num_, sample_info_ in test_construal_info_.items():
                # print_dict.update({(base_construal_name_, test_construal_name_, sample_num_): abs(sample_info_['log_likelihood_diff'])})
                print_dict.update({(base_construal_name_, test_construal_name_, sample_num_): sample_info_['log_likelihood']})
        print(dict(sorted(print_dict.items(), key=lambda item: item[1])))

Scene: tfrecord-00101-of-01000_71
{((10,), (0, 1, 5, 9, 10, 42, 43), 0): tensor(42.2862, grad_fn=<MulBackward0>), ((10,), (1,), 0): tensor(43.8516, grad_fn=<MulBackward0>), ((10,), (5,), 0): tensor(52.2410, grad_fn=<MulBackward0>), ((10,), (10,), 0): tensor(56.0830, grad_fn=<MulBackward0>), ((10,), (0,), 0): tensor(56.1282, grad_fn=<MulBackward0>), ((10,), (42,), 0): tensor(56.1282, grad_fn=<MulBackward0>), ((10,), (43,), 0): tensor(56.1282, grad_fn=<MulBackward0>), ((10,), (9,), 0): tensor(56.1312, grad_fn=<MulBackward0>), ((10,), (0, 1, 5, 9, 10, 42, 43), 1): tensor(88.5552, grad_fn=<MulBackward0>), ((10,), (1,), 1): tensor(91.9477, grad_fn=<MulBackward0>), ((10,), (5,), 1): tensor(109.8547, grad_fn=<MulBackward0>), ((10,), (10,), 1): tensor(118.1341, grad_fn=<MulBackward0>), ((10,), (0,), 1): tensor(118.3411, grad_fn=<MulBackward0>), ((10,), (42,), 1): tensor(118.3411, grad_fn=<MulBackward0>), ((10,), (43,), 1): tensor(118.3411, grad_fn=<MulBackward0>), ((10,), (9,), 1): tensor(118.

In [9]:
### Inference Logic ###

# |Get probability of lambda values
get_constral_heurisrtic_values_partial = partial(get_constral_heurisrtic_values, env=env, 
                                                 train_loader=train_loader, default_values=default_values)
p_lambda = {}
curr_heuristic_params = deepcopy(heuristic_params)
for curr_lambda in np.linspace(0,1,11):
    curr_lambda = curr_lambda.item()
    curr_heuristic_params["ego_distance"] = curr_lambda
    curr_heuristic_values = get_constral_heurisrtic_values_partial(heuristic_params=curr_heuristic_params)
    p_lambda[curr_lambda] = {}

    for scene_name, sampled_construals in construal_action_likelihoods.items():
        p_lambda[curr_lambda][scene_name] = {}
        for base_construal, base_construal_info in sampled_construals.items():
            p_lambda[curr_lambda][scene_name][base_construal] = []
            for test_construal, test_construal_info in base_construal_info.items():
                curr_p_lambda = []
                for sample_num, sample_info in test_construal_info.items():
                    p_a = torch.exp( -1*sample_info['log_likelihood'] ).item()
                    construal_heur_value = curr_heuristic_values[scene_name][test_construal]
                    curr_p_lambda.append(p_a*construal_heur_value)
                curr_p_lambda = np.prod([val for val in curr_p_lambda if val > 0])
                p_lambda[curr_lambda][scene_name][base_construal].append(curr_p_lambda)
            p_lambda[curr_lambda][scene_name][base_construal] = sum(p_lambda[curr_lambda][scene_name][base_construal])

Processing Waymo batches: 100%|██████████| 2/2 [00:00<00:00,  7.58it/s]


In [10]:
# |Get product over lambda probability across sampled construals
lamda_inference = {}
for curr_lambda, scene_info in p_lambda.items():
    lamda_inference[curr_lambda] = np.prod([val for scene_name, construal_info in scene_info.items() for val in construal_info.values() if val > 0])
lamda_inference

{0.0: np.float64(1.7664971882420559e-90),
 0.1: np.float64(1.6895007158807105e-90),
 0.2: np.float64(1.6121704657147238e-90),
 0.30000000000000004: np.float64(1.5348708221155833e-90),
 0.4: np.float64(1.4579589072949806e-90),
 0.5: np.float64(1.3817798359033603e-90),
 0.6000000000000001: np.float64(1.306662264813328e-90),
 0.7000000000000001: np.float64(1.232914353799306e-90),
 0.8: np.float64(1.160820236991477e-90),
 0.9: np.float64(1.090637085030683e-90),
 1.0: np.float64(1.0225928149304267e-90)}

In [ ]:
### Convert to pandas table and save ###
import pandas as pd

construal_action_likelihoods_df = {(scene,baseC,testC,sample): construal_action_likelihoods[scene][baseC][testC][sample]['log_likelihood'].item()
                                        for scene in construal_action_likelihoods.keys() 
                                        for baseC in construal_action_likelihoods[scene].keys() 
                                        for testC in construal_action_likelihoods[scene][baseC].keys() 
                                        for sample in construal_action_likelihoods[scene][baseC][testC].keys()}

multi_index = pd.MultiIndex.from_tuples(construal_action_likelihoods_df.keys(), names=['scene', 'base_construal', 'test_construal', 'sample'])
construal_action_likelihoods_df = pd.DataFrame(construal_action_likelihoods_df.values(), index=multi_index)
construal_action_likelihoods_df.columns = ['-log_likelihood']

construal_action_likelihoods_summarydf = construal_action_likelihoods_df.groupby(level=(0,1,2)).mean().sort_values(by='-log_likelihood', ascending=True).\
                                            groupby(level=(0,1)).head(5).sort_index(level=(0,1), sort_remaining=False)


In [81]:
construal_action_likelihoods_df.to_csv(out_dir + "construal_action_likelihoods.tsv", sep="\t", index=True, header=True)
construal_action_likelihoods_summarydf.to_csv(out_dir + "construal_action_likelihoods_summary.tsv", sep="\t", index=True, header=True)

In [80]:
construal_action_likelihoods_summarydf

-log_likelihood
scene                       base_construal test_construal                           
tfrecord-00101-of-01000_71  (10,)          (0, 1, 5, 9, 10, 42, 43)        65.420681
                                           (1,)                            67.899654
                                           (5,)                            81.047890
                                           (10,)                           87.108541
                                           (42,)                           87.234642
                            (42,)          (0, 1, 5, 9, 10, 42, 43)        59.722807
                                           (1,)                            61.587799
                                           (5,)                            78.954990
                                           (10,)                           84.919094
                                           (42,)                           85.054754
tfrecord-00117-of-01000_240 (16,)          (0,)                           163.441360
                                           (16,)                          163.441360
                                           (17,)                          163.441360
                                           (15,)                          163.441360
                                           (13,)                          163.441360
                            (17,)          (17,)                          152.360428
                                           (13,)                          152.360428
                                           (0,)                           152.360428
                                           (14,)                          152.360428
                                           (16,)                          152.360428
tfrecord-00143-of-01000_179 (1,)           (1,)                           110.036697
                                           (0,)                           129.262020
                                           (20,)                          129.262020
                                           (19,)                          129.262020
                                           (17,)                          129.262020
                            (6,)           (12,)                          113.709408
                                           (11,)                          115.033791
                                           (8,)                           116.049282
                                           (10,)                          117.051777
                                           (7,)                           117.055065
tfrecord-00144-of-01000_130 (0,)           (0, 7, 59)                      85.030964
                                           (7,)                            85.030964
                                           (0,)                            85.515448
                                           (59,)                           85.515448
                            (59,)          (0, 7, 59)                      79.572937
                                           (7,)                            79.572937
                                           (0,)                            80.045065
                                           (59,)                           80.045065

In [87]:
# |Save dataframe as latex table
print(construal_action_likelihoods_summarydf.to_latex(index=True))

\begin{tabular}{lllr}
\toprule
 &  &  & -log_likelihood \\
scene & base_construal & test_construal &  \\
\midrule
\multirow[t]{10}{*}{tfrecord-00101-of-01000_71} & \multirow[t]{5}{*}{(10,)} & (0, 1, 5, 9, 10, 42, 43) & 65.420681 \\
 &  & (1,) & 67.899654 \\
 &  & (5,) & 81.047890 \\
 &  & (10,) & 87.108541 \\
 &  & (42,) & 87.234642 \\
\cline{2-4}
 & \multirow[t]{5}{*}{(42,)} & (0, 1, 5, 9, 10, 42, 43) & 59.722807 \\
 &  & (1,) & 61.587799 \\
 &  & (5,) & 78.954990 \\
 &  & (10,) & 84.919094 \\
 &  & (42,) & 85.054754 \\
\cline{1-4} \cline{2-4}
\multirow[t]{10}{*}{tfrecord-00117-of-01000_240} & \multirow[t]{5}{*}{(16,)} & (0,) & 163.441360 \\
 &  & (16,) & 163.441360 \\
 &  & (17,) & 163.441360 \\
 &  & (15,) & 163.441360 \\
 &  & (13,) & 163.441360 \\
\cline{2-4}
 & \multirow[t]{5}{*}{(17,)} & (17,) & 152.360428 \\
 &  & (13,) & 152.360428 \\
 &  & (0,) & 152.360428 \\
 &  & (14,) & 152.360428 \\
 &  & (16,) & 152.360428 \\
\cline{1-4} \cline{2-4}
\multirow[t]{10}{*}{tfrecord-00143-of